In [1]:
import nltk

In [2]:
#nltk.download_shell()

In [3]:
messages = [line.strip() for line in open('smsspamcollection/SMSSpamCollection')]

In [4]:
#Printing some sample messages
for mess_no, message in enumerate(messages[:10]):
    print(mess_no,message)
    print('\n')

0 ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...


1 ham	Ok lar... Joking wif u oni...


2 spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's


3 ham	U dun say so early hor... U c already then say...


4 ham	Nah I don't think he goes to usf, he lives around here though


5 spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv


6 ham	Even my brother is not like to speak with me. They treat me like aids patent.


7 ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune


8 spam	WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code 

In [5]:
import pandas as pd

In [6]:
messages = pd.read_csv('smsspamcollection/SMSSpamCollection',sep='\t', names=['label','message'])

In [7]:
messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
messages.describe() 

,label,message
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [9]:
messages['length'] = messages['message'].apply(len)

In [10]:
messages.head()

,label,message,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [11]:
import string

In [12]:
from nltk.corpus import stopwords

In [13]:
#Remove punctuations, stopwords and returns list of cleaned text words
def text_process(mess):
    nopunc = [char for char in mess if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [14]:
messages['message'].head().apply(text_process)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: message, dtype: object

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
bow_transformer = CountVectorizer(analyzer=text_process).fit(messages['message'])

In [17]:
print(len(bow_transformer.vocabulary_))

11425


In [18]:
#printing sample bag of words transformer with random message
sample = messages['message'][3]

In [19]:
print(sample)

U dun say so early hor... U c already then say...


In [20]:
bow_sample = bow_transformer.transform([sample])

In [21]:
print(bow_sample)

  (0, 4068)	2
  (0, 4629)	1
  (0, 5261)	1
  (0, 6204)	1
  (0, 6222)	1
  (0, 7186)	1
  (0, 9554)	2


In [22]:
print(bow_sample.shape)

(1, 11425)


In [23]:
bow_transformer.get_feature_names()[9554]

'say'

### Applying bag of words for all messages

In [24]:
messages_bow = bow_transformer.transform(messages['message'])

In [25]:
print(messages_bow.shape)

(5572, 11425)


### Term frequency and Inverse document frequency

In [26]:
from sklearn.feature_extraction.text import TfidfTransformer

In [27]:
tfidf_transformer = TfidfTransformer().fit(messages_bow)

In [28]:
tfidf_sample = tfidf_transformer.transform(bow_sample)

In [29]:
print(tfidf_sample)

  (0, 9554)	0.5385626262927564
  (0, 7186)	0.4389365653379857
  (0, 6222)	0.3187216892949149
  (0, 6204)	0.29953799723697416
  (0, 5261)	0.29729957405868723
  (0, 4629)	0.26619801906087187
  (0, 4068)	0.40832589933384067


In [30]:
messages_tfidf = tfidf_transformer.transform(messages_bow)

### Importing Naive Bayes

In [31]:
from sklearn.naive_bayes import MultinomialNB

In [32]:
spam_detect_model = MultinomialNB().fit(messages_tfidf,messages['label'])

In [33]:
spam_detect_model.predict(tfidf_sample)[0]

'ham'

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
msg_train,msg_test,label_train,label_test = train_test_split(messages['message'],messages['label'],test_size=0.3)

### Pipeline

In [36]:
from sklearn.pipeline import Pipeline

In [37]:
pipeline = Pipeline([
    ('bow',CountVectorizer(analyzer=text_process)),
    ('tfidf',TfidfTransformer()),
    ('classifier',MultinomialNB())
])

In [38]:
#Training
pipeline.fit(msg_train,label_train)

Pipeline(memory=None,
     steps=[('bow', CountVectorizer(analyzer=<function text_process at 0x7f10718ed840>,
        binary=False, decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=None, min_df=1, ngram_range=(1, 1), preprocessor=No...f=False, use_idf=True)), ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [39]:
#Predictions
predictions = pipeline.predict(msg_test)

### Classification Report

In [40]:
from sklearn.metrics import classification_report

In [41]:
print(classification_report(label_test,predictions))

              precision    recall  f1-score   support

         ham       0.95      1.00      0.98      1454
        spam       1.00      0.68      0.81       218

   micro avg       0.96      0.96      0.96      1672
   macro avg       0.98      0.84      0.89      1672
weighted avg       0.96      0.96      0.96      1672

